In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.xception import preprocess_input
import numpy as np
from io import BytesIO
from urllib import request
from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [2]:
model = keras.models.load_model("dogs_cats_10_0.687.h5")

converter = tf.lite.TFLiteConverter.from_keras_model(model)
lite_model = converter.convert()
with open("dogs_cats_10_0.687.lite", 'wb') as f:
    f.write(lite_model)

2021-12-02 14:00:24.261056: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-02 14:00:24.871898: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/z0/gxg1dpkx0f9_gx0q14d8k2kc0000gn/T/tmpfittm0cy/assets


2021-12-02 14:00:27.672216: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:363] Ignored output_format.
2021-12-02 14:00:27.672234: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:366] Ignored drop_control_dependency.
2021-12-02 14:00:27.673114: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /var/folders/z0/gxg1dpkx0f9_gx0q14d8k2kc0000gn/T/tmpfittm0cy
2021-12-02 14:00:27.674573: I tensorflow/cc/saved_model/reader.cc:107] Reading meta graph with tags { serve }
2021-12-02 14:00:27.674591: I tensorflow/cc/saved_model/reader.cc:148] Reading SavedModel debug info (if present) from: /var/folders/z0/gxg1dpkx0f9_gx0q14d8k2kc0000gn/T/tmpfittm0cy
2021-12-02 14:00:27.683010: I tensorflow/cc/saved_model/loader.cc:210] Restoring SavedModel bundle.
2021-12-02 14:00:27.826787: I tensorflow/cc/saved_model/loader.cc:194] Running initialization op on SavedModel bundle at path: /var/folders/z0/gxg1dpkx0f9_gx0q14d8k2kc0000gn/T/tmpfittm0cy
2021-12

In [3]:
interpreter = tf.lite.Interpreter(model_path="dogs_cats_10_0.687.lite")
interpreter.allocate_tensors()
output_details = interpreter.get_output_details()
print(output_details)

[{'name': 'StatefulPartitionedCall:0', 'index': 13, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [4]:
img = prepare_image(
    download_image('https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'), 
    (150, 150)
)
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)

print(X[0][0][0][0])

0.41176474


In [5]:
# Preprocess Image
img = prepare_image(
    download_image('https://upload.wikimedia.org/wikipedia/commons/9/9a/Pug_600.jpg'), 
    (150, 150)
)
x = np.array(img)
X = np.array([x])
X = preprocess_input(X)

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="dogs_cats_10_0.687.lite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test model on random input data.
input_shape = input_details[0]['shape']
interpreter.set_tensor(input_details[0]['index'], X)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.97031206]]
